In [ ]:
import numpy as np
import math
from scipy.optimize import fsolve
class Params():
def __init__(self):

"""
Experiment Modes:
The modes listed below are test cases that were used in order to
verify the single-layer model with the results from Courtier et
al.
please note,
- if slm = ’off’, tlm is assumed to be ’on’.
- slm must be ’on’ for test_case_slm to function.
- to replicate the results from Courtier et al. single-layer
model use Beer-Lambert optical model.
"""

slm = ’on’
test_case_slm = ’on’

""" Choose Optical Model:
- ’beer-lambert’
- ’transfer-matrix’
"""
self.model = ’beer-lambert’

"""
Parameter Inputs
- Each parameter is defined as an instance of the parameters class
to allow for easy access through parameter objects.
"""

""" Tolerance and Resolution Parameters
N: number of grid points - also used to calculate NE and NH for
three-layer model
rtol: relative tolerance for Radau integrator timestep
atol: absolute tolerance for Radau integrator timestep
"""
self.N = 400
self.rtol = 1e-6
self.atol = 1e-10

""" J-V Scan Parameters

Vi: initial applied voltage/preconditioning voltage (V)
Vf: final applied voltage (V)
scan_rate: (V/s)
"""
self.Vi = 1.2
self.Vf = 0
self.scan_rate = 0.3

""" Constants

q: proton charge (C)
kB: Boltzmann constant (eVK-1)
T: Temperature (K)
eps0: permittivity of free space (Fm-1)
Fph: incident photon flux (m-2s-1)
Vt: thermal voltage (V)
"""

self.q = 1.60217646e-19
self.kB = 8.61733035e-5
self.T = 298.0
self.eps0 = 8.85418782e-12
self.Fph = 1.4e21
self.Vt = self.kB*self.T

""" Perovskite Layer Parameter Inputs

b: perovskite layer thickness (m)
epsp: permittivity of perovskite (Fm-1)
alpha: perovskite absorption coefficient (m-1)
Ec: conduction band minimum (eV)
Ev: valence band maximum (eV)
Dn: perovskite electron diffusion coefficient (m2s-1)
Dp: perovskite hole diffusion coefficient (m2s-1)
gc: conduction band density of states (m-3)
gv: valence band density of states (m-3)
"""

self.b = 600e-9
self.epsp = 24.1*self.eps0
self.alpha = 1.3e7
self.Ec = -3.7
self.Ev = -5.4
self.Dn = 1.7e-4
self.Dp = 1.7e-4
self.gc = 8.1e24
self.gv = 5.8e24


""" Ion Parameter Inputs

N0: density of ion vacancies (m-3)
D: diffusivity relation
DIinf: high temperature vacancy diffusion coefficient (m2s-1)
EAI: iodide vacancy activation energy (eV)
DI: diffusion coefficient for iodide ions (m2s-1)
"""

self.N0 = 1.6e25
self.D = lambda Dinf, EA: Dinf*np.exp(-EA/(self.kB*self.T))
self.DIinf = 6.5e-8
self.EAI = 0.58
self.DI = self.D(self.DIinf, self.EAI)


#Three-Layer Model Parameter Inputs

""" ETL Parameter Inputs

dE: effective doping density of ETL (m-3)
gcE: effective conduction band density of states in ETL (m-3)
EcE: conduction band minimum in ETL (eV)
bE: ETL layer thickness (m)
epsE: permittivity of ETL (Fm-1)
DE: electron diffusion coefficient in ETL (m2s-1)
"""
self.dE = 1e24
self.gcE = 5e25
self.EcE = -4.0
self.bE = 100e-9
self.epsE = 10*self.eps0
self.DE = 1e-5

""" HTL Parameter Inputs
dH: effective doping density of HTL (m-3)
gvH: effective valence band density of states in HTL (m-3)
EvH: valence band minimum in HTL (eV)
bH: HTL layer thickness (m)
epsH: permittivity of HTL (Fm-1)
DH: hole diffusion coefficient in HTL (m3s-1)
"""

self.dH = 1e24
self.gvH = 5e25
self.EvH = -5.1
self.bH = 200e-9
self.epsH = 3*self.eps0
self.DH = 1e-6


#Beer-Lambert Law Generation
self.Y = 3.7
self.w = 2.4


#Bulk Recombination Parameters

tn: SRH - electron pseudo-lifetime (s)
tp: SRH - hole pseudo-lifetime (s)
beta: bimolecular recombination rate (m3s-1)
self.tn = 3e-9
self.tp = 3e-7
self.beta = 0.0

""" Interface Recombination Parameters

betaE: ETL/perovskite bimolecular recombination rate (m3s-1)
betaH: perovskite/HTL bimolecular recombination rate (m3s-1)
vnE: SRH - electron recombination velocity - ETL (ms-1)
vpE: SRH - hole recombination velocity - ETL (ms-1)
vnH: SRH - electron recombination velocity - HTL (ms-1)
vpH: SRH - hole recombination velocity - HTL (ms-1)
"""
self.betaE = 0.0
self.betaH = 0.0
self.vnE = 1e5
self.vpE = 10
self.vnH = 0.1
self.vpH = 1e5

#PARAMETER INPUTS COMPLETE - Alteration to the following script may impact the accuracy of the simulation.

""" Perovskite Calculated Parameters & Non-dimensionalization

Eg: bandgap (eV)
LD: debye length (m)
lam: non-dimensional debye length parameter
lam2: non-dimensional debye length parameter squared
ni: intrinsic carrier density (m-3)
delta: ratio of electron and ion densities
chi: ratio of hole and electron densities
G0: typical rate of photogeneration (m-3s-1)
Tion: if DI !=0 then characteristic ionic timescale (s)
if DI = 0 then characteristic electronic timescale (s)
sigma: ratio of carrier and electronic timescales
Kp: hole current parameter
Kn: electron current parameter
Upsilon: Beer-Lambert photogeneration parameter
"""
self.Eg = self.Ec - self.Ev
self.LD = np.sqrt(self.Vt*self.epsp/(self.q*self.N0))
self.lam = self.LD/self.b
self.lam2 = self.lam**2
self.ni = np.sqrt(self.gc*self.gv)*np.exp(-self.Eg/(2*self.Vt))
self.delta = self.dE/self.N0
self.chi = self.dH/self.dE
self.G0 = np.array((self.Fph/self.b))*(1-np.exp(-self.alpha*self.b))

if self.DI != 0:
    self.Tion = self.b/self.DI*np.sqrt(self.Vt*self.epsp/(self.q*self.N0))
else:
    self.Tion = self.dE/self.G0
    self.sigma = self.dE/(self.G0*self.Tion)
    self.Kp = self.Dp*self.dH/(self.G0*self.b**2)
    self.Kn = self.Dn*self.dE/(self.G0*self.b**2)
    self.Upsilon = self.alpha*self.b

""" Energy Level Parameters
EfE: ETL workfunction (eV)
EfH: HTL workfunction (eV)
Vbi: built-in voltage (V)
pbi: non-dimensional built-in voltage
"""
self.EfE = self.EcE - self.Vt*np.log(self.gcE/self.dE)
self.EfH = self.EvH + self.Vt*np.log(self.gvH/self.dH)
self.Vbi = self.EfE - self.EfH
self.pbi = self.Vbi/self.Vt

""" Interface Parameters
kE: ratio between electron densities across ETL/perovskite interface
kH: ratio between hole densities across perovskite/HTL interface
n0: electron density in perovskite (m-3)
p0: hole density in perovskite (m-3)
"""
self.kE = self.gc/self.gcE*np.exp((self.EcE-self.Ec)/self.Vt)
self.kH = self.gv/self.gvH*np.exp((self.Ev-self.EvH)/self.Vt)
self.n0 = self.kE*self.dE
self.p0 = self.kH*self.dH

"""
Single-Layer Model Conditions
- the single-layer model uses a different non-dimensionalization
method than the three-layer model
- this condition modifies the few parameters that differ from the
three-layer model
"""
if slm == ’on’:
    #Single-Layer Model Non-Dimensionalized Parameters
    self.delta = self.Fph*self.b/(self.DE*self.N0)
    self.sigma = self.DI*self.b/(self.DE*self.LD)
    self.k_p = self.Dp/self.DE
    self.k_n = self.Dn/self.DE
elif test_case_slm == ’off’:
    #Do nothing


""" Dirichlet Boundary Conditions

n_bar: ETL interface electron density
p_bar: HTL interface hole density
"""
self.n_bar = self.n0*self.DE/(self.Fph*self.b)
self.p_bar = self.p0*self.DE/(self.Fph*self.b)

""" Scan-rate scaling

ion_mobility_factor: typical ion mobility used to non-dimensionalize
the scan rate
"""
self.ion_mobility_factor= 2.4025533333333337e-16

#Non-Dimensional Scan Parameters
self.psi_scan_rate = self.scan_rate*(self.LD*self.b/self.
ion_mobility_factor)/self.Vt
self.phi_i = self.Vi/self.Vt
self.phi_f = self.Vf/self.Vt
self.phi_precondition = self.phi_i
self.tf_scan = np.abs((self.phi_i - self.phi_f)/self.psi_scan_rate)


# Test Case Conditions

if test_case_slm == ’on’:
    self.pbi = 40
    self.DI = 2.4025533333333337e-16
    self.delta = 2.1*10e-7
    self.sigma = 5.8*10e-10
    self.n_bar = 20
    self.p_bar = 0.3
    self.lam = 2.4*10e-4
    self.lam2 = self.lam**2
    self.phi_precondition = 46.729618965853994
    self.phi_f = 0.0
    self.psi_scan_rate = 14.2
    self.tf_scan = np.abs((self.phi_precondition - self.phi_f)/self.
    psi_scan_rate)
    #self.tf_forward_scan = self.tf_reverse_scan
elif test_case_slm == ’off’:
    #Do nothing


""" ETL & HTL Parameters

wE: relative width of ETL
wH: relative width of HTL
KE: ETL electron current parameter
KH: HTL hole current parameter
rE: relative ETL permittivity
rH: relative HTL permittivity
lamE2: relative ETL Debye length parameter squared
lamE: relative ETL Debye length parameter
lamH2: relative HTL Debye length parameter squared
lamH: relative HTL Debye length parameter
OmegaE: ETL charge density parameter
OmegaH: HTL charge density parameter
"""
self.wE = self.bE/self.b
self.wH = self.bH/self.b
self.KE = self.DE*self.Kn/self.Dn
self.KH = self.DH*self.Kp/self.Dp
self.rE = self.epsE/self.epsp
self.rH = self.epsH/self.epsp
self.lamE2 = self.rE*self.N0/self.dE*self.lam2
self.lamE = np.sqrt(self.lamE2)
self.lamH2 = self.rH*self.N0/self.dH*self.lam2
self.lamH = np.sqrt(self.lamH2)
self.OmegaE = np.sqrt(self.N0/(self.rE*self.dE))
self.OmegaH = np.sqrt(self.N0/(self.rH*self.dH))



""" Bulk Recombination Parameters

ni2: non-dimensional intrinsic carrier density squared
brate: bimolecular recombination rate constant
gamma: rate constant for SRH recombination
tor: ratio of SRH carrier lifetimes
tor3: constant from deep trap approximation
- gamma, tor, tor3 = 0 if no bulk SRH recombination
"""
self.ni2 = self.ni**2/(self.dE*self.dH)
self.brate = self.beta*self.dE*self.dH/self.G0
if self.tp>0 and self.tn>0:
    self.gamma = self.dH/(self.tp*self.G0)
    self.tor = self.tn*self.dH/(self.tp*self.dE)
    self.tor3 = (self.tn+self.tp)*self.ni/(self.tp*self.dE)
else:
    [self.gamma, self.tor, self.tor3] = [0,0,0]


""" Interface Recombination Parameters

brateE: ETL bimolecular recombination rate constant
brateH: HTL bimolecular recombination rate constant
gammaE: ETL rate constant for SRH recombination
torE: ratio of SRH carrier lifetimes
torE3: constant from deep trap state approximation
- gammeE, torE, torE3 = 0 if no ETL/perovskite interface
recombination
gammaH: HTL rate constnat for SRH recombination
torH: ratio of SRH carrier lifetimes
torH3: constant from deep trap state approximation
- gammaH, torH, torH3 = 0 if no perovskite/HTL interface
recombination
"""
self.brateE = self.betaE*self.dE*self.dH/(self.b*self.G0)

self.brateH = self.betaH*self.dE*self.dH/(self.b*self.G0)

if self.vpE>0 and self.vnE>0:
    self.gammaE = self.dH*self.vpE/(self.b*self.G0)
    self.torE = self.dH*self.vpE/(self.dE*self.vnE)
    self.torE3 = (1/self.kE+self.vpE/self.vnE)*self.ni/self.dE
else:
    [self.gammaE, self.torE, self.torE3] = [0,0,0]
if self.vnH>0 and self.vpH>0:
    self.gammaH = self.dE*self.vnH/(self.b*self.G0)
    self.torH = self.dE*self.vnH/(self.dH*self.vpH)
    self.torH3 = (1/self.kH+self.vnH/self.vpH)*self.ni/self.dH
else:
    [self.gammaH, self.torH, self.torH3] = [0,0,0]



""" Spatial Discretization Parameters

X: percentage of grid points within ionic Debye length of the
interface
NE: number of grid points in ETL
NH: number of grid points in HTL
"""
self.X = 0.2
self.tanhfun = lambda x, st: (math.tanh(st*(2*x-1))/math.tanh(st)+1)/2

def func(st):
    return self.lam - self.tanhfun(self.X, st)
self.st = float(fsolve(func, 2))
self.A = lambda b: (math.tanh(self.st*(1-2/self.N))-(1-np.double(b))*math.tanh(self.st))/np.double(b)
self.NE = np.int(np.round(2/(1-math.atanh(self.A(self.wE))/self.st)))
self.NH = np.int(np.round(2/(1-math.atanh(self.A(self.wH))/self.st))